Metaclasse
- Cria classe de forma dinâmica
- Pode ser útil na criação de frameworks onde as classes precisam ter um comportamento específico

In [7]:
class MeuMeta(type):
    def __new__(cls, nome, bases, dct):
        dct['novo_atributo']='valor adicionado pela metaclasse'
        return super().__new__(cls,nome,bases,dct)

In [10]:
class MinhaClasse(metaclass=MeuMeta):
    pass

In [16]:
obj=MinhaClasse()
obj.novo_atributo


'valor adicionado pela metaclasse'

In [ ]:
class ValidadorMeta(type):
    def __new__(cls, nome, bases, dct):
        validacoes = dct.get('validacoes', {})
        
        for attr, tipo in validacoes.items():
            if not callable(tipo):
                raise TypeError(f'O tipo de validação para {attr} deve ser uma função')
            def valida_func(self,value, attr=attr, tipo=tipo):
                if not isinstance(value, tipo):
                    raise ValueError(f'Valor inválido para {attr}. Deve ser do tipo {tipo.__name__}')
                
                self.__dict__[attr] = value
                
            valida_func.__name__=f'set_{attr}'
            dct[f"set_{attr}"] = valida_func
                
        return super().__new__(cls,nome,bases,dct)

    @staticmethod
    def valida(funcao):
        def wrapper(*args, **kwargs):
            print('Validando...')
            return funcao(*args, **kwargs)
        return wrapper

In [13]:
class Usuario(metaclass=ValidadorMeta):
    validacoes = {
        'nome': str,
        'idade': int,
    }
    
    @ValidadorMeta.valida
    def __init__(self, nome, idade):
        self.set_nome(nome)
        self.set_idade(idade)

In [18]:
try:
    user=Usuario('João', 25)
    print(f"Nome: {user.nome}, Idade: {user.idade}")
    user.set_idade("trinta")
except ValueError as e:
    print(e)
    pass

Validando...


AttributeError: 'Usuario' object has no attribute 'nome'